In [1]:
import pandas as pd
from datetime import timedelta

In [44]:
df = pd.read_csv('KC_case_data .csv', parse_dates=['date'])

In [3]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [4]:
df.shape

(2747968, 8)

In [5]:
df.nunique()

date                91
event                7
purchase_sum      8775
os_name              2
device_id       190884
gender               2
city                 2
utm_source           7
dtype: int64

In [6]:
df.utm_source.unique()

array(['-', 'vk_ads', 'referal', 'facebook_ads', 'google_ads',
       'instagram_ads', 'yandex-direct'], dtype=object)

In [34]:
df['month'] = df['date'].dt.to_period('M')

In [8]:
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads,2020-01
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal,2020-01
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads,2020-01
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads,2020-01


In [9]:
# MAU
df.groupby('month', as_index=False).device_id.nunique()

,month,device_id
0,2020-01,99161
1,2020-02,75032
2,2020-03,74623


In [10]:
# app installs
actions = df.groupby(['month', 'event'], as_index=False).device_id.nunique()
actions.head()

,month,event,device_id
0,2020-01,app_install,80297
1,2020-01,app_start,99161
2,2020-01,choose_item,81872
3,2020-01,purchase,39012
4,2020-01,register,42946


### Conversion from install to purchase

In [11]:
installs = df[df.event == 'app_install'].copy()
installs['cohort'] = installs.date

In [12]:
df = df.merge(installs[['device_id', 'cohort']], on='device_id')
df.head()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,cohort
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-,2020-01,2020-01-01
1,2020-01-01,search,NaN,android,669460,female,Moscow,-,2020-01,2020-01-01
2,2020-01-03,app_start,NaN,android,669460,female,Moscow,-,2020-01,2020-01-01
3,2020-01-03,choose_item,NaN,android,669460,female,Moscow,-,2020-01,2020-01-01
4,2020-01-03,search,NaN,android,669460,female,Moscow,-,2020-01,2020-01-01


In [13]:
filtered_df = df[df['event'].isin(['app_install', 'purchase'])]
filtered_df.tail()

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,cohort
2263005,2020-03-31,app_install,NaN,android,908531,male,Saint-Petersburg,yandex-direct,2020-03,2020-03-31
2263010,2020-03-31,app_install,NaN,ios,3925522,female,Saint-Petersburg,instagram_ads,2020-03,2020-03-31
2263012,2020-03-31,app_install,NaN,android,6369878,female,Saint-Petersburg,google_ads,2020-03,2020-03-31
2263017,2020-03-31,app_install,NaN,android,18983670,female,Moscow,instagram_ads,2020-03,2020-03-31
2263022,2020-03-31,app_install,NaN,android,29262521,male,Moscow,yandex-direct,2020-03,2020-03-31


In [15]:
filtered_df.shape

(269822, 10)

In [16]:
def check_within_7_days(group):
    if 'app_install' in group['event'].values and 'purchase' in group['event'].values:
        app_install_date = group[group['event'] == 'app_install']['date'].iloc[0]
        purchase_date = group[group['event'] == 'purchase']['date'].iloc[0]
        return (purchase_date - app_install_date) <= timedelta(days=7)
    return False

# Group by 'uid' and apply the function to filter rows
result = filtered_df.groupby('device_id').filter(check_within_7_days)
result

,date,event,purchase_sum,os_name,device_id,gender,city,utm_source,month,cohort
13,2020-01-07,purchase,269.0,android,669460,female,Moscow,-,2020-01,2020-01-01
22,2020-01-09,purchase,4687.5,android,669460,female,Moscow,-,2020-01,2020-01-01
46,2020-01-01,app_install,NaN,android,669460,female,Moscow,-,2020-01,2020-01-01
50,2020-01-01,purchase,329.5,ios,833621,male,Moscow,vk_ads,2020-01,2020-01-01
62,2020-01-11,purchase,601.0,ios,833621,male,Moscow,referal,2020-01,2020-01-01
...,...,...,...,...,...,...,...,...,...,...
2262989,2020-03-31,app_install,NaN,android,14204302,male,Moscow,vk_ads,2020-03,2020-03-31
2262993,2020-03-31,purchase,7542.0,android,18219505,female,Saint-Petersburg,-,2020-03,2020-03-31
2262996,2020-03-31,app_install,NaN,android,18219505,female,Saint-Petersburg,-,2020-03,2020-03-31
2263002,2020-03-31,purchase,654.0,android,908531,male,Saint-Petersburg,yandex-direct,2020-03,2020-03-31


In [54]:
cohorts = df[df.event == 'app_install'].groupby('cohort', as_index=False).agg(users=('device_id', 'nunique'))

In [55]:
paying_users = result[result.event == 'purchase'].groupby('cohort', as_index=False).agg(pu=('device_id', 'nunique'))

In [56]:
cohorts = cohorts.merge(paying_users, on='cohort')
cohorts['conversion'] = round(100 * cohorts.pu / cohorts.users, 1)

In [57]:
cohorts

,cohort,users,pu,conversion
0,2020-01-01,3579,1408,39.3
1,2020-01-02,3144,1186,37.7
2,2020-01-03,2402,834,34.7
3,2020-01-04,1831,639,34.9
4,2020-01-05,1671,587,35.1
...,...,...,...,...
86,2020-03-27,1199,277,23.1
87,2020-03-28,1091,209,19.2
88,2020-03-29,1117,223,20.0
89,2020-03-30,994,204,20.5


#### С какого платного маркетингового канала пришло больше всего новых пользователей? 

In [6]:
df.groupby('utm_source', as_index=False).agg(users=('device_id','nunique')).sort_values('users', ascending=False)

,utm_source,users
0,-,52273
6,yandex-direct,40712
2,google_ads,38096
5,vk_ads,34086
3,instagram_ads,31048
1,facebook_ads,25959
4,referal,15926


#### When do we lose our clients

search  --> choose_item --> tap_basket --> register

In [59]:
search = df[df.event == 'search'].device_id.nunique()
choose_item = df[df.event == 'choose_item'].device_id.nunique()
tap_basket = df[df.event == 'tap_basket'].device_id.nunique()
register = df[df.event == 'register'].device_id.nunique()

In [60]:
print(f'search: {search}, --> {round(100*choose_item/search)}% --> choose_item: {choose_item}, \
--> {round(100*tap_basket/choose_item)}% --> tap_basket: {tap_basket}, \
--> {round(100*register/tap_basket)}% --> register: {register}') 

search: 184488, --> 84% --> choose_item: 155691, --> 81% --> tap_basket: 125414, --> 62% --> register: 78310


#### Conversion into a purchase

In [77]:
app_opens = df[df.utm_source != '-'].query('event == "app_start"') \
            .groupby('utm_source', as_index=False).agg(users=('device_id', 'nunique'))
purchases = df[df.utm_source != '-'].query('event == "purchase"') \
            .groupby('utm_source', as_index=False).agg(pu=('device_id', 'nunique'))
conversion = app_opens.merge(purchases, on='utm_source')
conversion['conversion'] = 100 * conversion.pu / conversion.users
conversion.sort_values('conversion')

,utm_source,users,pu,conversion
5,yandex-direct,40707,12028,29.547744
1,google_ads,38092,11339,29.767405
2,instagram_ads,31045,10762,34.665808
0,facebook_ads,25957,9017,34.738221
4,vk_ads,34079,12364,36.280407
3,referal,15924,6362,39.952273


### Пользователи, пришедшие с какого канала, имеют медианный первый чек выше

In [97]:
purch = df[df.utm_source != '-'].query('event == "purchase"').sort_values('date')

In [99]:
first_purch = purch.groupby("device_id").first().reset_index()

In [103]:
first_purch.groupby('utm_source').agg(meadian_purch=('purchase_sum', 'median')) \
            .sort_values('meadian_purch', ascending=False).reset_index()

,utm_source,meadian_purch
0,referal,395.50
1,instagram_ads,393.50
2,yandex-direct,393.00
3,vk_ads,392.50
4,google_ads,389.50
5,facebook_ads,389.25


#### ROMI

In [107]:
source_revenue = df[df.utm_source != '-'].groupby('utm_source', as_index=False) \
                        .agg(revenue=('purchase_sum', 'sum'), users=('device_id', 'nunique'))
source_revenue

,utm_source,revenue,users
0,facebook_ads,12249901.0,25959
1,google_ads,12868276.0,38096
2,instagram_ads,14546969.0,31048
3,referal,8837044.5,15926
4,vk_ads,16389652.5,34086
5,yandex-direct,13915368.0,40712


In [126]:
referal_cost = (source_revenue[source_revenue.utm_source == 'referal'].users * 100).to_list()[0]

In [127]:
def costs(source):
    if source == 'facebook_ads':
        return 8590498
    elif source == 'google_ads':
        return 10534878
    elif source == 'instagram_ads':
        return 8561626
    elif source == 'vk_ads':
        return 9553531
    elif source == 'yandex-direct':
        return 10491707
    elif source == 'referal':
        return referal_cost

In [128]:
source_revenue['cost'] = source_revenue.utm_source.apply(costs)
source_revenue

,utm_source,revenue,users,cost
0,facebook_ads,12249901.0,25959,8590498
1,google_ads,12868276.0,38096,10534878
2,instagram_ads,14546969.0,31048,8561626
3,referal,8837044.5,15926,1592600
4,vk_ads,16389652.5,34086,9553531
5,yandex-direct,13915368.0,40712,10491707


In [130]:
source_revenue['romi'] = (source_revenue.revenue - source_revenue.cost)/source_revenue.cost
source_revenue.sort_values('romi', ascending=False)

,utm_source,revenue,users,cost,romi
3,referal,8837044.5,15926,1592600,4.548816
4,vk_ads,16389652.5,34086,9553531,0.715560
2,instagram_ads,14546969.0,31048,8561626,0.699090
0,facebook_ads,12249901.0,25959,8590498,0.425983
5,yandex-direct,13915368.0,40712,10491707,0.326321
1,google_ads,12868276.0,38096,10534878,0.221493
